In [17]:
import numpy as np
import gensim
from gensim.models import word2vec
import logging
#load pretrain vec

model=gensim.models.KeyedVectors.load_word2vec_format('/media/kwane/3160053c-937e-4de9-a540-b28bd2802040/kwane/NLP/data/word2vec.txt',binary=False) #GloVe Model

In [18]:
import numpy as np
path='./question/'

train = list(open(path+"train.txt", "r").readlines())
train = [s.strip() for s in train]
print len(train)
valid = list(open(path+"test.txt", "r").readlines())
valid = [s.strip() for s in valid]
print len(valid)
max_len=0
for i,line in enumerate(train+valid):
    label, _, _words = line.replace('\xf0', ' ').partition(' ')
    words = _words.strip().split()
    #print words,len(words)
    if len(words)>max_len:
        max_len=len(words)

print max_len

5452
500
37


In [19]:
#corpus
import torch
import logging
import argparse
import os
import numpy as np
from const import *

word2vec=np.random.uniform(-0.25,0.25,(3,4))

def word2idx(sents, word2idx):
    return [[word2idx[w] if w in word2idx else UNK for w in s] for s in sents]


class Dictionary(object):
    def __init__(self, word2idx={}, idx_num=0):
        self.word2idx = word2idx
        self.idx = idx_num

    def _add(self, word):
        if self.word2idx.get(word) is None:
            self.word2idx[word] = self.idx
            self.idx += 1

    def _convert(self):
        self.idx2word = {v: k for k, v in self.word2idx.items()}

    def __len__(self):
        return self.idx

    def __str__(self):
        return "%s(size = %d)".format(self.__class__.__name__, len(self.idx))


class Words(Dictionary):
    def __init__(self):
        word2idx = {
            WORD[PAD]: PAD,
            WORD[UNK]: UNK
        }
        super(Words,self).__init__(word2idx=word2idx, idx_num=len(word2idx))

    def __call__(self, sents):
        words = set([word for sent in sents for word in sent])
        for word in words:
            self._add(word)


class Labels(Dictionary):
    def __init__(self):
        super(Labels,self).__init__()

    def __call__(self, labels):
        _labels = set(labels)
        for label in _labels:
            self._add(label)


class Corpus(object):
    def __init__(self, path, save_data, max_len):
        self.train = os.path.join(path, "train.txt")
        self.valid = os.path.join(path, "test.txt")
        self._save_data = save_data

        self.w = Words()
        self.l = Labels()
        self.max_len = max_len

    def parse_data(self, _file, is_train=True, fine_grained=False):
        """
        fine_grained: Whether to use the fine-grained (50-class) version of TREC
                or the coarse grained (6-class) version.
        """
        _sents, _labels = [], []
        for sentence in open(_file):
            label, _, _words = sentence.replace('\xf0', ' ').partition(' ')
            label = label.split(":")[0] if not fine_grained else label

            words = _words.strip().split()

            if len(words) > self.max_len:
                words = words[:self.max_len]

            _sents += [words]
            _labels += [label]
        if is_train:
            self.w(_sents)
            self.l(_labels)
            self.train_sents = _sents
            self.train_labels = _labels
        else:
            self.valid_sents = _sents
            self.valid_labels = _labels

    def save(self):
        self.parse_data(self.train)
        self.parse_data(self.valid, False)
        #print self.l.word2idx,[self.l.word2idx[l] for l in self.train_labels]
        def mask_(i):#mask for rnn
            mask=np.zeros(self.max_len, dtype=int)
            mask[i-1]=1
            return mask
        def mask_all(i):#mask for bi-rnn & cnn
            mask=np.zeros(self.max_len, dtype=int)
            mask[:i]=np.ones(i, dtype=int)
            return mask
        data = {
            'max_len': self.max_len,
            'dict': {
                'train': self.w.word2idx,
                'vocab_size': len(self.w),
                'label': self.l.word2idx,
                'label_size': len(self.l),
            },
            'train': {
                'src': word2idx(self.train_sents, self.w.word2idx),
                'label': [self.l.word2idx[l] for l in self.train_labels],
                'train_mask':[mask_(len(s)) for s in self.train_sents],
                'train_mask_all':[mask_all(len(s)) for s in self.train_sents]
            },
            'valid': {
                'src': word2idx(self.valid_sents, self.w.word2idx),
                'label': [self.l.word2idx[l] for l in self.valid_labels],
                'valid_mask':[mask_(len(s)) for s in self.valid_sents],
                'valid_mask_all':[mask_all(len(s)) for s in self.valid_sents]
            }
        }

        torch.save(data, self._save_data)
        print('Finish dumping the data to file - [{}]'.format(self._save_data))
        print('words length - [{}]'.format(len(self.w)))
        print('label size - [{}]'.format(len(self.l)))


if __name__ == "__main__":

    
    corpus = Corpus(path, path+"corpus.pt", max_len)
    corpus.save()


Finish dumping the data to file - [./data/question/corpus.pt]
words length - [9450]
label size - [7]


In [20]:


data = torch.load(path+'corpus.pt')
vocab_size=data['dict']['vocab_size']
train_dic_num=data['train']['src']
valid_dic_num=data['valid']['src']

word2vec=word2vec=np.zeros((vocab_size,300))
#train
for i,line in enumerate(train):
    line=str.lower(line)
    label, _, _words = line.replace('\xf0', ' ').partition(' ')
    words = _words.strip().split()
    words = words[:max_len]
    for j,word in enumerate(words):
        #print i,j,train_dic_num[i][j]
        try:
            word2vec[train_dic_num[i][j],:]=model[word]
        except:
            word2vec[train_dic_num[i][j],:]=np.random.uniform(-0.25,0.25,300)
#valid
for i,line in enumerate(valid):
    line=str.lower(line)
    label, _, _words = line.replace('\xf0', ' ').partition(' ')
    words = _words.strip().split()
    words = words[:max_len]
    for j,word in enumerate(words):
        try:
            word2vec[valid_dic_num[i][j],:]=model[word]
        except:
            word2vec[valid_dic_num[i][j],:]=np.random.uniform(-0.25,0.25,300)
print train_dic_num[0],train_dic_num[1]
np.save(path+'pretrained_wordvec',word2vec)
#x=np.load('./data/pretrained_wordvec.npy')
#print np.size(word2vec),word2vec

[1176, 5282, 4343, 3323, 5070, 7908, 2485, 649, 8755, 4563] [7943, 718, 4870, 2765, 5126, 7076, 1945, 4563]
